<a href="https://colab.research.google.com/github/KlaidasKaralevicius/NLP_lab2/blob/main/lab2_Klaidas_Karalevicius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os, re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import random

In [62]:
import keras
print('Keras version:', keras.__version__)
keras.utils.set_random_seed(226)
random.seed(226)

Keras version: 3.4.1


In [8]:
!pip install markovify -q
import markovify
print('Markovify version:', markovify.__version__)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.1 MB/s eta 0:00:00
Markovify version: 0.9.4


In [30]:
!pip install num2words -q
from num2words import num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 4.3 MB/s eta 0:00:00


In [66]:
!pip install pronouncing -q
import pronouncing
print('Pronouncing version:', pronouncing.__version__)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 12.9 MB/s eta 0:00:00
Pronouncing version: 0.2.0


# 1. Įkelti dainų tekstus


In [46]:
!wget https://raw.githubusercontent.com/KlaidasKaralevicius/NLP_lab2/refs/heads/main/Arknights.txt
artist_file = 'Arknights.txt'
with open(artist_file, 'r') as f:
  print(f.read(500))
  #print(len(f.read()))

--2024-11-12 22:59:05--  https://raw.githubusercontent.com/KlaidasKaralevicius/NLP_lab2/refs/heads/main/Arknights.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128963 (126K) [text/plain]
Saving to: ‘Arknights.txt’

Arknights.txt       100%[===================>] 125.94K  --.-KB/s    in 0.03s   

2024-11-12 22:59:05 (4.51 MB/s) - ‘Arknights.txt’ saved [128963/128963]

Echoing reality
Are your memories of me?
Floating through a state
half asleep, never awake (never awake)
I won't breathe in eternity
Carrying your secrets
feeling close to who you are
You're just beyond my reach
A passing breeze
And your final moments set me free
Gentle flowing thoughts in time
Sometimes yours and sometimes mine
I can't tell where this one ends
My ever lost, forever friend


# 2. Duomenų normalizavimas

In [38]:
def normalize_word_line(line: str) -> list[str]:
  line = re.sub(r"\[.*?\]", "", line)
  row = [x.lower() for x in re.findall(r"\w+'?\w*", line)]
  new_row = []
  for word in row:
    numbers = re.findall(r"\d+", word)
    for n in numbers:
      word = word.replace(n, num2words(int(n)))
    new_row.append(word)
  return new_row

# 3. Markovo grandinės

In [63]:
with open(os.path.join(artist_file), 'r') as f:
  lyrics = f.read()

# Random seed neužfiksuotas, todėl visi gausite skirtingus rezultatus
markov_model = markovify.NewlineText(lyrics)

for i in range(5):
    print(markov_model.make_sentence())

Oh... As winter blows in to freeze up in the depths of my code
Under cover of the nightmares from my past
will I get to the sky is turning red, but I SHOW THEM ALL!
I know you know I strike back.
I think that way.


##3. Ritmo ištraukimas iš duomenų

In [64]:
def n_syllables(word_line: list[str]):
    vowels = 'aeiouy'
    syllable_count = 0

    for word in word_line:
        for i, char in enumerate(word):
            if char in vowels:
                if (i == 0) or (word[i-1] not in vowels):
                    syllable_count += 1
        word_vowels_count = sum([x in vowels for x in word])
        if word_vowels_count == 0:  # abreviaturos
            syllable_count = len(word)  # pvz. "NLP" skaitosi kaip "en-el-pi"
        elif word_vowels_count > 1 and (word[-1] == 'e') and (word[-2] not in vowels):  # paskutinė "e" dažnai nesakoma anglų k., bet "ie", "ee" ištariama
            syllable_count -= 1

    return syllable_count

In [65]:
with open(os.path.join(artist_file), 'r') as f:
  lyrics = f.read()
sentence = normalize_word_line(lyrics.splitlines()[10])
print(sentence)
print(f'N syllables: {n_syllables(sentence)}')

['gentle', 'flowing', 'thoughts', 'in', 'time']
N syllables: 6


Tačiau mes nenorime naudoti bele kokius žodžius, o norime prioretizuoti žodžius iš jau esamų atlikėjo dainų. Todėl reikia sukonstruoti rimuojančių žodžių žodyną.

In [71]:
def get_rhyme(line: list[str]) -> str:
    last_word = re.sub('\W+', '', line[-1])
    all_rhymes = pronouncing.rhymes(last_word)
    if all_rhymes:
        rhyming_ends = [x[-2:] for x in all_rhymes]
        most_common_rhyme = max(set(rhyming_ends), key=rhyming_ends.count)
    else:
        most_common_rhyme = last_word[-2:]
    return most_common_rhyme


def get_rhyme_list(normalized_lyrics: list[list[str]]):
  rhyme_set = set()
  for row in normalized_lyrics:
    most_common_rhyme = get_rhyme(row)
    rhyme_set.add(most_common_rhyme)

  sorted_rhyme_set = sorted(list(rhyme_set), key=lambda x: x[-1])
  return sorted_rhyme_set


with open(os.path.join(artist_file), 'r') as f:
  lyrics = f.read()
lyrics = [normalize_word_line(x) for x in lyrics.splitlines()]
lyrics = [x for x in lyrics if x]
rhymes = get_rhyme_list(lyrics)
print(f'Collected {len(rhymes)} rhymes')

Collected 147 rhymes


## 4. Duomenų rinkinio paruošimas

Modelis negalės tiesiogiai operuoti skiemenimis, todėl turime naudoti skaičius:

In [73]:
def get_rhyme_float(line: list[str], rhyme_list: list[str]) -> float | None:
  rhyme = get_rhyme(line)
  if rhyme in rhyme_list:
    return rhyme_list.index(rhyme) / len(rhyme_list)
  else:
    return None


test = [markov_model.make_sentence() for _ in range(5)]
for line in test:
  rhyme_float = get_rhyme_float(normalize_word_line(line), rhymes)
  print(f'Rhyme float: {rhyme_float:.2f} \tLine:', line)

Rhyme float: 0.25 	Line: My dignity, just let it come, let it go– no, I can't see.
Rhyme float: 0.20 	Line: And if you wait.
Rhyme float: 0.87 	Line: Are you used to know.
Rhyme float: 0.46 	Line: We are never backing down, hear us when we fall
Rhyme float: 0.71 	Line: Dashing through the walls.


In [74]:
def get_random_lines(markov_model, n_rows: int) -> list[list[str]]:
  lines = []
  last_words = []

  while len(lines) < n_rows:
    line = markov_model.make_sentence(max_overlap_ratio=.49, tries=100)
    # nenorime gauti tuščios eilutės ar jau turimos eilutės
    if (line is not None) and (line not in lines):
      last_word = normalize_word_line(line)[-1]
      # nenorime kad dažnai pasikartotų tas pats žodis eilutės gale
      if last_words.count(last_word) < 3:
        lines.append(normalize_word_line(line))
        last_words.append(last_word)

  return lines


def get_line_features(line: list[str], rhyme_list: list[str]) -> tuple:
  return (line, n_syllables(line), get_rhyme_float(line, rhyme_list)) # n_syllables(line) turi buti float ne int


test_lines = get_random_lines(markov_model, 2)
for line in test_lines:
  print(get_line_features(line, rhymes))

(['on', 'the', 'fence', 'is', 'not', 'a', 'part', 'of', 'me', "they'll", 'ricochet'], 13, 0.9659863945578231)
(['i', 'just', 'need', 'the', 'wind', 'to', 'carry', 'through', 'i', 'used', 'to', 'keeping', 'it', 'all'], 17, 0.4557823129251701)


Konstruojame duomenų rinkinį:

In [75]:
def build_dataset(lines: list[list[str]], rhyme_list: list[str]):
	features = [get_line_features(x, rhyme_list) for x in lines]
	x_data, y_data = [], []

  # turėsime standartinę struktūrą kai eilutės rimuojasi po 4 grupėje
	# pirmos dvi eilutės bus pradinės savybės, antros dvi eilutės - prognozuojamos
	for i in range(len(features) - 3):
		# duomenyse liks tik eilučių savybes, todėl visur [1:]
		line1, line2 = features[i    ][1:], features[i + 1][1:]
		line3, line4 = features[i + 2][1:], features[i + 3][1:]
		x_data.append(np.array([line1, line2]))
		y_data.append(np.array([line3, line4]))
	return np.array(x_data), np.array(y_data)


dataset = build_dataset(lyrics, rhymes)
len(dataset[0]), len(dataset[1])

(3978, 3978)

In [76]:
# pirmo mėginio pvz.
dataset[0][0], dataset[1][0]

(array([[5.        , 0.25170068],
        [7.        , 0.25170068]]),
 array([[ 5.        ,  0.19727891],
        [11.        ,  0.19047619]]))

## 5. RNN modelio inicializavimas

Mūsų modelis turės prognozuoti naujų 4 eilučių savybes gaunant senas 4 eilutes.

In [77]:
def create_lstm(depth: int):

  keras.backend.clear_session()  # pašaliname tarpinių modelių likučius
  keras.utils.set_random_seed(812)

  model = keras.Sequential(name='LSTM-based_lyrics_generator')
  model.add(keras.layers.Input((2, 2)))
  model.add(keras.layers.LSTM(4, return_sequences=True))
  for i in range(depth):
    model.add(keras.layers.LSTM(8, return_sequences=True)) # LSTM keisti GRU
  model.add(keras.layers.LSTM(2, return_sequences=True))

  model.compile(
      optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
      loss='mse')

  return model

model = create_lstm(depth=1)
model.summary()

Model: "LSTM-based_lyrics_generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 2, 4)                │             112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 2, 8)                │             416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 2, 2)                │              88 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 616 (2.41 KB)

 Trainable params: 616 (2.41 KB)

 Non-trainable params: 0 (0.00 B)

Turime **dviejų modelių sistemą**, todėl būtina parašyti funkcijas kad modeliai galėtų tarpusavyje bendrauti.

In [78]:
def compose(starting_input: np.ndarray, rnn_model, n_line_groups: int):
	prev_vectors = starting_input
	final_vectors = []
	for i in range(n_line_groups):
		final_vectors.append(rnn_model.predict(prev_vectors).flatten().reshape(1, 2, 2))
		prev_vectors = final_vectors[-1]
	return final_vectors


def last_word_compare(prev_lines: list[list[str]], new_line: list[str], penalty: float = 0.2) -> float:
	sum_penalty = 0.0
	for line in prev_lines:
		if line[-1] == new_line[-1]:
			sum_penalty += penalty
	return sum_penalty


def calculate_score(features, n_syllables, rhyme, penalty: float, rhyme_list, maxsyllables):
	desired_n_syllables = features[0] * maxsyllables
	desired_rhyme = features[1] * len(rhyme_list)
	syllable_score = - abs(float(desired_n_syllables) - float(n_syllables))
	rhyme_score = abs(float(desired_rhyme) - float(rhyme))
	score = 1.0 + syllable_score + rhyme_score - penalty
	return score


def vectors_into_song(vectors, generated_lyrics, rhyme_list, maxsyllables: int):
	song = []
	generated_features = [get_line_features(x, rhyme_list) for x in generated_lyrics]

	vector_halves = []
	for vector in vectors:
		vector_halves.extend(vector[0].tolist())

	for vector in vector_halves:
		scorelist = []

		for (line, n_syllables, rhyme) in generated_features:
			if len(song) != 0:
				penalty = last_word_compare(song, line)
			else:
				penalty = 0

			total_score = calculate_score(vector, n_syllables, rhyme, penalty, rhyme_list, maxsyllables)
			scorelist.append([line, total_score])

		# randame eilutę su aukščiausiu įvertinimu
		best_line_index = np.argmax([float(x[1]) for x in scorelist])
		best_line = scorelist[best_line_index][0]
		song.append(best_line)

		# pašaliname šią eilutę iš likusių eilučių sąrašo
		generated_features = [x for x in generated_features if x[0] != best_line]

	return [' '.join(x) for x in song]

# kokie žodžiai bus sugeneruoti be modelio apmokymo?
start = np.array([dataset[0][0]])
vectors = compose(start, model, 4)
some_lyrics = get_random_lines(markov_model, 20)
vectors_into_song(vectors, some_lyrics, rhymes, maxsyllables=12)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


["i don't know how to make it through",
 'come on show me i can hold up',
 'tell me can you show me what you want',
 "you glow like your heart on your hands won't do",
 'so long did i do when i feel the fog',
 'but how long can you feel the heat in your eye',
 'grinding down the only thing that you speak of',
 "then there's the light of your era era"]

## 6. Modelio apmokymas

In [79]:
x_data, y_data = dataset

model.fit(
    x_data, y_data,
    batch_size=2,
    epochs=2
)

Epoch 1/2
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 29.2386
Epoch 2/2
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 27.2845


In [80]:
# kokie žodžiai bus sugeneruoti po modelio apmokymo?
start = np.array([dataset[0][0]])
vectors = compose(start, model, 4)
some_lyrics = get_random_lines(markov_model, 20)
vectors_into_song(vectors, some_lyrics, rhymes, maxsyllables=12)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


['break down the only thing that you embrace me',
 'i just wanna look at this world feels so unknown',
 'let your heart on my tongue',
 'always finding time for the rest of my own path',
 "cause i've laid it out in the face of danger",
 "feels like i'm trapped inside trapped inside my",
 "maybe i won't see the smoke in the aftermath oh",
 "right back at you like a beacon now you're not getting in"]

Sudedame viską į vientą vietą:

In [82]:
artist_file = 'Arknights.txt'
with open(os.path.join(artist_file), 'r') as f:
    raw_lyrics = f.read()

markov_model = markovify.NewlineText(raw_lyrics)

lyrics = [normalize_word_line(x) for x in raw_lyrics.splitlines()]
lyrics = [x for x in lyrics if x]
rhymes = get_rhyme_list(lyrics)
print(f'Collected {len(rhymes)} rhymes')

x_data, y_data = build_dataset(lyrics, rhymes)

model = create_lstm(depth=4)
print(model.summary())

model.fit(
    x_data, y_data,
    batch_size=2,
    epochs=10
)

start_i = np.random.choice(range(len(x_data)))
start = np.array([x_data[start_i]])
vectors = compose(start, model, 4)
some_lyrics = get_random_lines(markov_model, 200)
vectors_into_song(vectors, some_lyrics, rhymes, maxsyllables=12)

Collected 147 rhymes


Model: "LSTM-based_lyrics_generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 2, 4)                │             112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 2, 8)                │             416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 2, 8)                │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 2, 8)                │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 2, 8)                │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 2, 2)                │              88 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,248 (8.78 KB)

 Trainable params: 2,248 (8.78 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 29.1125
Epoch 2/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 27.2843
Epoch 3/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 27.2842
Epoch 4/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - loss: 27.2842
Epoch 5/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 27.2842
Epoch 6/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 27.2842
Epoch 7/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 27.2841
Epoch 8/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 27.2841
Epoch 9/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 27.2841
Epoch 10/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 27.2841
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 866ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


['walk the edge but i know move forward',
 'the heart needs time to rest up and saw the real world',
 'this will never give up this whole town',
 "then there's the light of your era era",
 "i don't know how to get it over",
 'oh but the time has come to terms with the left right',
 "there's a plan you can't burn us out",
 "and take it all i'll take us to the other side"]